## ComputerVision and Convolutional Neural Networks in TensorFlow

### This part is covering:

* Getting a dataset to work with
* Architecture of a *Convolutional Neural Network (CNN)* with Tensorflow
* An end-to-end binary image classification problem
* Steps in modelling with **CNN's**
 * Creating a CNN, compiling a model, fitting a model, evaluating a model
* An end-to-end multi-class image classification problem
* Making predictions on our own custom images
